In [1]:
import numpy as np
import torch
import shap
from ChestWorld import ChestWorld
from dqn_agent_new import DqnAgentNew

pygame 2.6.0 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Config, load agent and env
max_steps=1_000_000

# ChestWorld
render_mode=None
size=6
agentSpawn = None
q_values=None
maxSteps=200
stepLoss=-1/maxSteps # min reward should be -1
wallCoordinates=None
randomWalls=0
chestCoordinates=None
keyCoordinates=None
randomchests=3
randomkeys=5
chestReward=1/min(randomchests, randomkeys) # max reward should be 1

# Agent
batch_size=64
lr=0.001
gamma=0.95
epsilon_start=1
epsilon_min=0.05
epsilon_decay=200_000 # 50_000 at 3000 episodes
tau=0.0005 # Was 0.005
replayBuffer=100_000

env = ChestWorld(render_mode=None, size=size, agentSpawn=agentSpawn, q_values=q_values, stepLoss=stepLoss, maxSteps=maxSteps, wallCoordinates=wallCoordinates, randomWalls=randomWalls, chestCoordinates=chestCoordinates, keyCoordinates=keyCoordinates, chestReward=chestReward, randomchests=randomchests, randomkeys=randomkeys)
observation, _ = env.reset()
agent = DqnAgentNew(env.action_space, observation,
    batch_size=batch_size,
    lr=lr,
    gamma=gamma,
    epsilon_start=epsilon_start,
    epsilon_min=epsilon_min,
    epsilon_decay=epsilon_decay,
    tau=tau,
    replayBuffer=replayBuffer,
    wandb=False)

Action space: 4
Observation space: 6
Observation space[0]: 6
Action space: 4
Observation space: 6
Observation space[0]: 6


In [3]:
agent.replayBuffer=replayBuffer
agent.load_model_weights(f"C:/Projects/public/XAI_NTNU/modelsToEval/CW_3chests_5keys_6x6_10000000steps.pth")
agent.inference(env=env, max_steps=100_000, epsilon=epsilon_min)

action_direction = {0: "Right",1: "Down",2: "Left", 3: "Up"}

replayBuffer = agent.replay_buffer
replayBufferStates = [replayBuffer[i][0] for i in range(len(replayBuffer))]
replayBufferStates = torch.stack(replayBufferStates)
replayBufferStates = replayBufferStates.squeeze(1)
print(f"replayBufferStates shape: {replayBufferStates.shape}")


Model loaded: C:/Projects/public/XAI_NTNU/modelsToEval/CW_3chests_5keys_6x6_10000000steps.pth
1%, time elapsed: 0 minutes and 0.59 seconds, it may finish around: 2024-11-20 15:00:02
10%, time elapsed: 0 minutes and 5.89 seconds, it may finish around: 2024-11-20 15:00:03
50%, time elapsed: 0 minutes and 27.62 seconds, it may finish around: 2024-11-20 14:59:59
99%, time elapsed: 0 minutes and 54.53 seconds, it may finish around: 2024-11-20 14:59:59
replayBufferStates shape: torch.Size([100000, 6, 6, 3])


In [4]:
model = agent.policy_net
# first 100 states
background = replayBufferStates[:10_000]
#with torch.no_grad():
print(f"Background shape: {background.shape}")

Background shape: torch.Size([10000, 6, 6, 3])


In [5]:
e = shap.DeepExplainer(model=model,
                       data=background,
                       session=None,
                       learning_phase_flags=None)

In [7]:
test_states = replayBufferStates[10_000:10_010]
with torch.no_grad():
    predictions = model(test_states).detach().numpy()
    argmax = [action_direction[np.argmax(prediction)] for prediction in predictions]
    print(f"Predictions: {predictions}")
    print(f"Argmax: {argmax}")

shap_values = e.shap_values(test_states) #, check_additivity=False)
"""AssertionError: The SHAP explanations do not sum up to the
model's output! This is either because of a rounding error or
because an operator in your computation graph was not fully
supported. If the sum difference of %f is significant compared
to the scale of your model outputs, please post as a github issue,
with a reproducible example so we can debug it. Used framework:
    pytorch - Max. diff: 0.07974889467004687 - Tolerance: 0.01"""
# check_additivity was set to False for this reason
# this error needs to be fixed since it gives a feulty result

shap_numpy = list(np.transpose(shap_values, (4, 0, 2, 1, 3)))
test_numpy = np.transpose(test_states.numpy(), (0, 2, 1, 3))

labels = np.array([[f"{action_direction[i]}: {predictions[j][i]:.3f}" for i in range(len(predictions[j]))] for j in range(len(predictions))])

# plot the feature attributions
shap.image_plot(shap_values=shap_numpy,
                pixel_values=test_numpy,
                labels=labels,
                true_labels=argmax,)
#                labelpad=0,
#                width=18)

Predictions: [[0.6630057  0.63042384 0.66851246 0.63307345]
 [0.7097063  0.65940773 0.7079278  0.66820216]
 [0.6630057  0.63042384 0.66851246 0.63307345]
 [0.7097063  0.65940773 0.7079278  0.66820216]
 [0.6630057  0.63042384 0.66851246 0.63307345]
 [0.7097063  0.65940773 0.7079278  0.66820216]
 [0.44284147 0.4169799  0.44803724 0.42762148]
 [0.1365298  0.10738182 0.08858541 0.12260336]
 [0.13616663 0.11486167 0.09829283 0.12203023]
 [0.14867774 0.13298857 0.11465088 0.13913205]]
Argmax: ['Left', 'Right', 'Left', 'Right', 'Left', 'Right', 'Left', 'Right', 'Right', 'Right']


AssertionError: The SHAP explanations do not sum up to the model's output! This is either because of a rounding error or because an operator in your computation graph was not fully supported. If the sum difference of %f is significant compared to the scale of your model outputs, please post as a github issue, with a reproducible example so we can debug it. Used framework: pytorch - Max. diff: 0.6342376136308303 - Tolerance: 0.01